In [1]:
import numpy as np
from feadme.parser import Template, Disk, Parameter, Line, Shape
from feadme.compose import evaluate_disk_model, disk_model
import numpyro

/Users/nmearl/code/feadme/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-12 01:51:21 | DEBUG    | Initializing...
2025-04-12 01:51:21 | DEBUG    | Backend: cpu
2025-04-12 01:51:21 | DEBUG    | Device count: 2
2025-04-12 01:51:21 | DEBUG    | Device: cpu
2025-04-12 01:51:21 | DEBUG    | Local device count: 2


In [2]:
disk_profile = Disk(
    name="halpha_disk",
    center=Parameter(name="center", distribution="normal", low=6562.819 - 5, high=6562.819 + 5, loc=6562.819, scale=1),
    inner_radius=Parameter(name="inner_radius", distribution="log_uniform", low=1e2, high=5e3),
    delta_radius=Parameter(name="delta_radius", distribution="log_uniform", low=5e2, high=1e5),
    inclination=Parameter(name="inclination", distribution="normal", low=0, high=np.pi / 2, loc=np.pi / 4, scale=np.pi / 8),
    sigma=Parameter(name="sigma", distribution="log_uniform", low=1e2, high=1e4),
    q=Parameter(name="q", distribution="uniform", low=0.5, high=4),
    eccentricity=Parameter(name="eccentricity", distribution="uniform", low=0, high=1),
    apocenter=Parameter(name="apocenter", distribution="normal", low=0, high=2 * np.pi, loc=np.pi, scale=np.pi / 4),
    scale=Parameter(name="scale", distribution="uniform", low=0, high=1),
    offset=Parameter(name="offset", distribution="uniform", low=0, high=0.05, fixed=True, value=0)
)

halpha_narrow_line = Line(
    name="halpha_narrow",
    shape=Shape.gaussian,
    center=Parameter(name="center", distribution="normal", low=6548, high=6568, loc=6562.819, scale=2, shared="halpha_disk"),
    amplitude=Parameter(name="amplitude", distribution="uniform", low=0, high=1),
    vel_width=Parameter(name="vel_width", distribution="log_uniform", low=1e1, high=1e3),
)

template = Template(
    name="test",
    redshift=0.0,
    mjd=0,
    data_path="",
    disk_profiles=[
        disk_profile,
    ],
    line_profiles=[],
)

param_mods = {
    "halpha_disk_center": 6562.819,
    "halpha_disk_inner_radius": 500,
    "halpha_disk_outer_radius": 2500,
    "halpha_disk_inclination": np.pi / 6,
    "halpha_disk_sigma": 800,
    "halpha_disk_q": 3,
    "halpha_disk_eccentricity": 0.3,
    "halpha_disk_apocenter": np.pi / 2,
    "halpha_disk_scale": 1,
    "halpha_disk_offset": 0,
}


wave = np.linspace(6300, 6800, 100)

In [3]:
# numpyro.render_model(disk_model, model_args=(template, wave, np.random.sample(wave.shape), np.random.sample(wave.shape),), filename="/Users/nmearl/Downloads/model_map.png")

In [4]:
_ = evaluate_disk_model(template, wave, param_mods)[0]

In [5]:
%%timeit

_ = evaluate_disk_model(template, wave, param_mods)[0]

637 μs ± 32.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
